In [2]:
# Import required libraries
import time
import random
from pymongo import MongoClient
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("📚 Libraries imported successfully!")

📚 Libraries imported successfully!


In [3]:
# MongoDB Configuration
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "online_retail_db"

# Connect to MongoDB
client = MongoClient(MONGO_URI, maxPoolSize=50, minPoolSize=10)
db = client[DB_NAME]
transaction_collection = db.transactions
customer_collection = db.customers

print("✅ Connected to MongoDB successfully!")
print(f"📊 Transaction documents: {transaction_collection.count_documents({})}")
print(f"👥 Customer documents: {customer_collection.count_documents({})}")

✅ Connected to MongoDB successfully!
📊 Transaction documents: 66
👥 Customer documents: 48


In [4]:
# Helper function to measure execution time
def measure_time(func, *args, **kwargs):
    """Measure execution time of a function"""
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return end_time - start_time, result

print("⏱️  Timing function ready!")

⏱️  Timing function ready!


In [11]:
# Get sample data for testing
sample_transaction = transaction_collection.find_one()
sample_customer = customer_collection.find_one()

test_transaction_no = sample_transaction['invoice_no']
test_customer_id = sample_customer['customer_id']
test_stock_code = sample_transaction['line_items'][0]['stock_code']

print(f"🧪 Test Transaction: {test_transaction_no}")
print(f"🧪 Test Customer: {test_customer_id}")
print(f"🧪 Test Stock Code: {test_stock_code}")

🧪 Test Transaction: 536365
🧪 Test Customer: 17850
🧪 Test Stock Code: 85123A


In [12]:
print("🚀 Testing CREATE Operations...")

def create_transaction_document():
    new_transaction = {
        'invoice_no': f'TEST_{int(time.time())}_{random.randint(1000,9999)}',
        'invoice_date': '2024-01-01',
        'customer_id': test_customer_id,
        'country': 'Test Country',
        'cancelled': False,
        'line_items': [
            {
                'line_id': 999999,
                'stock_code': 'TEST_ITEM',
                'description': 'Test Product',
                'quantity': 10,
                'unit_price': 15.99,
                'line_total': 159.90
            }
        ],
        'total_amount': 159.90
    }
    return transaction_collection.insert_one(new_transaction)

def create_customer_document():
    new_customer = {
        'customer_id': int(time.time()),
        'country': 'Test Country',
        'transactions': [
            {
                'invoice_no': f'TEST_{int(time.time())}',
                'invoice_date': '2024-01-01',
                'cancelled': False,
                'items': [
                    {
                        'stock_code': 'TEST_ITEM',
                        'description': 'Test Product',
                        'quantity': 5,
                        'unit_price': 20.0,
                        'total': 100.0
                    }
                ]
            }
        ],
        'total_spent': 100.0,
        'total_items_purchased': 5
    }
    return customer_collection.insert_one(new_customer)

# Test CREATE operations
create_results = []
for i in range(50):
    time_taken, _ = measure_time(create_transaction_document)
    create_results.append(('transaction_create', time_taken))
    
    time_taken, _ = measure_time(create_customer_document)
    create_results.append(('customer_create', time_taken))

# Calculate averages
avg_transaction_create = np.mean([t for op, t in create_results if op == 'transaction_create'])
avg_customer_create = np.mean([t for op, t in create_results if op == 'customer_create'])

print(f"✅ CREATE Operations Completed!")
print(f"   Transaction-centric: {avg_transaction_create:.6f}s")
print(f"   Customer-centric: {avg_customer_create:.6f}s")

🚀 Testing CREATE Operations...
✅ CREATE Operations Completed!
   Transaction-centric: 0.000392s
   Customer-centric: 0.000289s


In [13]:
print("🔍 Testing READ Operations...")

def read_transaction_by_invoice():
    return transaction_collection.find_one({'invoice_no': test_transaction_no})

def read_customer_by_id():
    return customer_collection.find_one({'customer_id': test_customer_id})

def read_transactions_by_country():
    return list(transaction_collection.find({'country': 'United Kingdom'}).limit(10))

def read_customers_by_country():
    return list(customer_collection.find({'country': 'United Kingdom'}).limit(10))

# Test READ operations
read_results = []
for i in range(50):
    # Simple reads
    time_taken, _ = measure_time(read_transaction_by_invoice)
    read_results.append(('transaction_read_simple', time_taken))
    
    time_taken, _ = measure_time(read_customer_by_id)
    read_results.append(('customer_read_simple', time_taken))
    
    # Complex reads
    time_taken, _ = measure_time(read_transactions_by_country)
    read_results.append(('transaction_read_complex', time_taken))
    
    time_taken, _ = measure_time(read_customers_by_country)
    read_results.append(('customer_read_complex', time_taken))

# Calculate averages
avg_transaction_read_simple = np.mean([t for op, t in read_results if op == 'transaction_read_simple'])
avg_customer_read_simple = np.mean([t for op, t in read_results if op == 'customer_read_simple'])
avg_transaction_read_complex = np.mean([t for op, t in read_results if op == 'transaction_read_complex'])
avg_customer_read_complex = np.mean([t for op, t in read_results if op == 'customer_read_complex'])

print(f"✅ READ Operations Completed!")
print(f"   Simple Read - Transaction: {avg_transaction_read_simple:.6f}s")
print(f"   Simple Read - Customer: {avg_customer_read_simple:.6f}s")
print(f"   Complex Read - Transaction: {avg_transaction_read_complex:.6f}s")
print(f"   Complex Read - Customer: {avg_customer_read_complex:.6f}s")

🔍 Testing READ Operations...
✅ READ Operations Completed!
   Simple Read - Transaction: 0.000667s
   Simple Read - Customer: 0.000424s
   Complex Read - Transaction: 0.000473s
   Complex Read - Customer: 0.000505s


In [14]:
print("✏️  Testing UPDATE Operations...")

def update_transaction():
    return transaction_collection.update_one(
        {'invoice_no': test_transaction_no},
        {'$set': {'country': 'Updated Country'}}
    )

def update_customer():
    return customer_collection.update_one(
        {'customer_id': test_customer_id},
        {'$set': {'country': 'Updated Country'}}
    )

def update_transaction_line_item():
    return transaction_collection.update_one(
        {'invoice_no': test_transaction_no, 'line_items.stock_code': test_stock_code},
        {'$set': {'line_items.$.quantity': 100}}
    )

def update_customer_transaction():
    return customer_collection.update_one(
        {'customer_id': test_customer_id, 'transactions.invoice_no': test_transaction_no},
        {'$set': {'transactions.$.items.0.quantity': 50}}
    )

# Test UPDATE operations
update_results = []
for i in range(50):
    # Simple updates
    time_taken, _ = measure_time(update_transaction)
    update_results.append(('transaction_update_simple', time_taken))
    
    time_taken, _ = measure_time(update_customer)
    update_results.append(('customer_update_simple', time_taken))
    
    # Nested updates
    time_taken, _ = measure_time(update_transaction_line_item)
    update_results.append(('transaction_update_nested', time_taken))
    
    time_taken, _ = measure_time(update_customer_transaction)
    update_results.append(('customer_update_nested', time_taken))

# Calculate averages
avg_transaction_update_simple = np.mean([t for op, t in update_results if op == 'transaction_update_simple'])
avg_customer_update_simple = np.mean([t for op, t in update_results if op == 'customer_update_simple'])
avg_transaction_update_nested = np.mean([t for op, t in update_results if op == 'transaction_update_nested'])
avg_customer_update_nested = np.mean([t for op, t in update_results if op == 'customer_update_nested'])

print(f"✅ UPDATE Operations Completed!")
print(f"   Simple Update - Transaction: {avg_transaction_update_simple:.6f}s")
print(f"   Simple Update - Customer: {avg_customer_update_simple:.6f}s")
print(f"   Nested Update - Transaction: {avg_transaction_update_nested:.6f}s")
print(f"   Nested Update - Customer: {avg_customer_update_nested:.6f}s")

✏️  Testing UPDATE Operations...
✅ UPDATE Operations Completed!
   Simple Update - Transaction: 0.000630s
   Simple Update - Customer: 0.000496s
   Nested Update - Transaction: 0.000480s
   Nested Update - Customer: 0.000325s


In [15]:
print("🧹 Cleaning up test data...")

# Delete test transactions and customers
transaction_collection.delete_many({'invoice_no': {'$regex': '^TEST_'}})
customer_collection.delete_many({'customer_id': {'$gte': 1700000000}})

print("✅ Test data cleaned up!")

🧹 Cleaning up test data...
✅ Test data cleaned up!


In [16]:
print("📊 Preparing results data...")

# Compile all results
results = {
    'create_transaction': avg_transaction_create,
    'create_customer': avg_customer_create,
    'read_simple_transaction': avg_transaction_read_simple,
    'read_simple_customer': avg_customer_read_simple,
    'read_complex_transaction': avg_transaction_read_complex,
    'read_complex_customer': avg_customer_read_complex,
    'update_simple_transaction': avg_transaction_update_simple,
    'update_simple_customer': avg_customer_update_simple,
    'update_nested_transaction': avg_transaction_update_nested,
    'update_nested_customer': avg_customer_update_nested
}

# Create DataFrame for visualization
df = pd.DataFrame([
    {'Operation': 'Create', 'Transaction_Centric': results['create_transaction'], 
     'Customer_Centric': results['create_customer']},
    {'Operation': 'Read Simple', 'Transaction_Centric': results['read_simple_transaction'], 
     'Customer_Centric': results['read_simple_customer']},
    {'Operation': 'Read Complex', 'Transaction_Centric': results['read_complex_transaction'], 
     'Customer_Centric': results['read_complex_customer']},
    {'Operation': 'Update Simple', 'Transaction_Centric': results['update_simple_transaction'], 
     'Customer_Centric': results['update_simple_customer']},
    {'Operation': 'Update Nested', 'Transaction_Centric': results['update_nested_transaction'], 
     'Customer_Centric': results['update_nested_customer']},
])

print("✅ Results data prepared!")
print("\n📈 Performance Summary:")
print(df.round(6))

📊 Preparing results data...
✅ Results data prepared!

📈 Performance Summary:
       Operation  Transaction_Centric  Customer_Centric
0         Create             0.000392          0.000289
1    Read Simple             0.000667          0.000424
2   Read Complex             0.000473          0.000505
3  Update Simple             0.000630          0.000496
4  Update Nested             0.000480          0.000325
